# Building a RAG pipeline with Llamaindex and Meltemi.

## Setting file directory and installing Llama-index modules.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/data

'Γενετική-μετασχηματιστική γραμματική - Βικιπαίδεια.pdf'  'Σύνταξη (γλωσσολογία) - Βικιπαίδεια.pdf'


In [ ]:
DATA_DIR="/content/drive/MyDrive/data"

In [ ]:
!pip install llama-index llama-index-readers-file llama-index-llms-openai llama-index-embeddings-huggingface llama-index-llms-litellm

## Loading required libraries

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.llms.litellm import LiteLLM

## Ingesting data

In [ ]:
documents = SimpleDirectoryReader(DATA_DIR).load_data()
len(documents)

4

In [ ]:
documents[0]

Document(id_='e36e5a4c-55ed-406d-8494-36067b48fa19', embedding=None, metadata={'page_label': '1', 'file_name': 'Γενετική-μετασχηματιστική γραμματική - Βικιπαίδεια.pdf', 'file_path': '/content/drive/MyDrive/data/Γενετική-μετασχηματιστική γραμματική - Βικιπαίδεια.pdf', 'file_type': 'application/pdf', 'file_size': 108823, 'creation_date': '2024-12-01', 'last_modified_date': '2024-12-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Γενετική-μετασχηματιστική γραμματική\nΣτη γλωσσολογία, μία μετασχηματιστική γραμματική ή αλλιώς γενετική-μετασχηματιστική\nγραμματική είναι η γενετική γραμματική, ειδικότερα μίας φυσικής γλώσσας, η οποία έχει αναπτυχθεί στις\nσυντακτικές δομές των γραμματικώ

## Chunking documents

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
)

nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
len(nodes)

7

In [ ]:
nodes[0]

TextNode(id_='eeb9cf13-63d8-42a8-ae0b-718ed876ae85', embedding=None, metadata={'page_label': '1', 'file_name': 'Γενετική-μετασχηματιστική γραμματική - Βικιπαίδεια.pdf', 'file_path': '/content/drive/MyDrive/data/Γενετική-μετασχηματιστική γραμματική - Βικιπαίδεια.pdf', 'file_type': 'application/pdf', 'file_size': 108823, 'creation_date': '2024-12-01', 'last_modified_date': '2024-12-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e36e5a4c-55ed-406d-8494-36067b48fa19', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'Γενετική-μετασχηματιστική γραμματική - Βικιπαίδεια.pdf', 'file_path': '/content/drive/MyDrive/data/Γενετική-μετασχηματιστική γραμματική - Βικιπα

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")
index = VectorStoreIndex(nodes, embed_model=embed_model)

In [ ]:
# gpt35 = LLamaIndexOpenAI(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")
# meltemi = LLamaIndexOpenAI(api_key=MELTEMI_API_KEY, base_url=MELTEMI_BASE_URL, model="meltemi-instruct-7b")

meltemi = LiteLLM(
    "hosted_vllm/meltemi-vllm",
    api_base="http://ec2-3-19-37-251.us-east-2.compute.amazonaws.com:4000/",
    api_key="sk-RYF0g_hDDIa2TLiHFboZ1Q",
)

In [ ]:
from llama_index.core.prompts import (
    SelectorPromptTemplate,
    PromptType,
    PromptTemplate,
    MessageRole,
    ChatPromptTemplate,
    ChatMessage,
)

QA_PROMPT = """BEGININPUT
{context_str}

ENDINPUT
BEGININSTRUCTION
{query_str}
ENDINSTRUCTION"""

QA_SYSTEM_PROMPT = """Είσαι ένας ειλικρινής και αμερόληπτος βοηθός που πάντα απαντάει με ακρίβεια σε αυτά που του ζητούνται.
Σου δίνονται δύο έγγραφα τα οποία βρίσκονται μεταξύ του BEGININPUT και του ENDINPUT.
Επίσης, σου δίνονται μεταδεδομένα για τα συγκεκριμένα έγγραφα μεταξύ του BEGINCONTEXT και του ENDCONTEXT.
Το βασικό κείμενο των εγγράφων βρίσκεται μεταξύ του ENDCONTEXT και του ENDINPUT.
Απάντησε στις οδηγίες του χρήστη που βρίσκονται μεταξύ του BEGININSTRUCTION και του ENDINSTRUCTION χρησιμοποιώντας μόνο το βασικό κείμενο
και τα μεταδεδομένα των εγγράφων που σου δίνονται παρακάτω. Αν οι οδηγίες που σου ζητάει ο χρήστης δεν μπορούν να απαντηθούν με το βασικό
κείμενο ή τα μεταδεδομένα των εγγράφων, ενημέρωσε τον χρήστη ότι δεν ξέρεις τη σωστή απάντηση."""

REFINE_PROMPT = """Η αρχική ερώτηση είναι η εξής: {query_str}
Έχεις δώσει την παρακάτω απάντηση: {existing_answer}
Έχεις την ευκαιρία να βελτιώσεις την προηγούμενη απάντηση (μόνο αν χρειάζεται) με τις παρακάτω νέες πληροφορίες.
------------
{context_str}
------------
Με βάση τις νέες πληροφορίες, βελτίωσε την απάντησή σου για να απαντά καλύτερα στην ερώτηση.
Αν οι πληροφορίες δεν είναι χρήσιμες, απλά επανάλαβε την προηγούμενη απάντηση.
Βελτιωμένη απάντηση: """

REFINE_CHAT_PROMPT = """Είσαι ένα έμπιστο σύστημα που απαντάει ερωτήσεις χρηστών. Λειτουργείς με τους εξής δύο τρόπους όταν βελτιώνεις υπάρχουσες απαντήσεις:
1. **Ξαναγράφεις** την απάντηση με βάση νέες πληροφορίες που σου παρέχονται
2. **Επαναλαμβάνεις** την προηγούμενη απάντηση αν δεν είναι χρήσιμες οι νέες πληροφορίες

Ποτέ δεν αναφέρεις την αρχική απάντηση ή το συγκείμενο απευθείας στην απάντησή σου.
Αν υπάρχει αμφιβολία για το τι πρέπει να απαντήσεις, απλά επανέλαβε την αρχική απάντηση.
Νέες πληροφορίες:
------------
{context_str}
------------
Ερώτηση: {query_str}
Αρχική απάντηση: {existing_answer}
Βελτιωμένη απάντηση: """


default_template = PromptTemplate(
    metadata={"prompt_type": PromptType.QUESTION_ANSWER},
    # template_vars=["context_str", "query_str"],
    template=QA_PROMPT,
)
chat_template = ChatPromptTemplate(
    metadata={"prompt_type": PromptType.CUSTOM},
    # template_vars=["context_str", "query_str"],
    message_templates=[
        ChatMessage(role=MessageRole.SYSTEM, content=QA_SYSTEM_PROMPT),
        ChatMessage(role=MessageRole.USER, content=QA_PROMPT),
    ],
)
text_qa_prompt = SelectorPromptTemplate(
    default_template=default_template, conditionals=[(lambda llm: True, chat_template)]
)

default_refine_template = PromptTemplate(
    metadata={"prompt_type": PromptType.REFINE},
    # template_vars=["query_str", "existing_answer", "context_str"],
    template=REFINE_PROMPT,
)
chat_refine_template = ChatPromptTemplate(
    metadata={"prompt_type": PromptType.CUSTOM},
    # template_vars=["context_str", "query_str", "existing_answer"],
    message_templates=[ChatMessage(role=MessageRole.USER, content=REFINE_CHAT_PROMPT)],
)
text_refine_prompt = SelectorPromptTemplate(
    default_template=default_refine_template,
    conditionals=[(lambda llm: True, chat_refine_template)],
)

In [ ]:
query_engine = index.as_query_engine(
    llm=meltemi, text_qa_prompt=text_qa_prompt, text_refine_prompt=text_refine_prompt
)

In [ ]:
response = query_engine.query("Ποιος είναι ο στόχος του γλωσσολόγου σύμφωνα με την γενετική μετασχηματιστική γραμματική;")
print(str(response))

Ο στόχος του γλωσσολόγου σύμφωνα με την γενετική μετασχηματιστική γραμματική είναι να συλλάβει και να περιγράψει τον μηχανισμό γένεσης των προτάσεων, συγκεκριμένα τη γλωσσική ικανότητα, η οποία περιλαμβάνει την περιγραφή της γλωσσικής επιτέλεσης, δηλαδή της ικανότητας παραγωγής λόγου σε πραγματικές περιστάσεις όπου μεσολαβούν διάφοροι ψυχολογικοί παράγοντες.


In [ ]:
response = query_engine.query("Πώς αιτιολογείται η ύπαρξη της καθολικής γραμματικής σύμφωνα με τον Noam Chomsky;")
print(str(response))

Η ύπαρξη της καθολική γραμματικής σύμφωνα με τον Noam Chomsky αιτιολογείται από την άποψη ότι η πρόταση έχει ιεραρχική δομή, η οποία μπορεί να αναπαρασταθεί στο πλαίσιο των γραμματικών φραστικής δομής. Οι κανόνες φραστικής δομής προσφέρουν το πλεονέκτημα της "αναδρομής", δηλαδή, της ιδιότητας των κανόνων να μπορούν να εφαρμόζονται απεριόριστα. Επιπλέον, η γενετική-μετασχηματιστική γραμματική, η οποία προτάθηκε από τον Chomsky, υποστηρίζει ότι η γλώσσα προϋπάρχουν ως εγγενή στοιχεία (δομές) στο μυαλό του παιδιού και ενεργοποιούνται στο γλωσσικό περιβάλλον όπου μεγαλώνει.
